In [17]:
import numpy as np
import pandas as pd
import json
import os
from tqdm import tqdm_notebook 


# Merge files

In [89]:
credit=pd.read_csv("credit_cleaned.csv")
collaterals=pd.read_csv("collaterals_optimized.csv").drop("Unnamed: 0",axis=1)
client=pd.read_csv("client_data_cleaned.csv")
overdue=pd.read_csv("overdue_with_new_features.csv")

In [91]:
credit_client=credit.merge(client,on="client_id")
credit_client_overdue=credit_client.merge(overdue,on="contract_id")
data=credit_client_overdue.merge(collaterals,how="left",on="contract_id")
data=data.fillna(0) ## not all contracts have collaterals

# New features

In [93]:
#Convert dates into datetime format
end=data.date_end.apply(lambda x: pd.to_datetime(x))
download=data.download_date.apply(lambda x: pd.to_datetime(x))
date_begin=data.date_begin.apply(lambda x: pd.to_datetime(x))
data["begin_datetime"]=date_begin
data["end_datetime"]=end
data["download_datetime"]=download

In [94]:
#Functions to claculate days 
def delta_exp(x):
    days=(x.end_datetime-x.download_datetime).days
    if days>0:
        return 0
    else:
        return 1 
    
def delta(x):
    days=(x.end_datetime-x.download_datetime).days
    return days 

def delta_begin(x):
    days=(x.download_datetime-x.begin_datetime).days
    return days 

In [95]:
#New features with days 
days=data.apply(lambda x: delta(x),axis=1)
exp=data.apply(lambda x: delta_exp(x),axis=1)
srok=data.apply(lambda x: delta_begin(x),axis=1)
data["duration of contract"]=srok
data["contract is expired"]=exp
data["Days till the end"]=days

In [103]:
#New feature with collateral and contract sum
data["dolya zaloga"]=data["summa zaloga"]/data["dog_summa_nt"]

In [ ]:
data.to_csv("data_final.csv")

# Final data for model

In [106]:
data_model=data.drop(['download_date', 'download_month', 'last_pay_date',
       'no', 'overdue_day','branch_code','depart_code','contract_manager_dep_code',
       'date_begin', 'date_end', 'client_id','begin_datetime','end_datetime', 'download_datetime','contract_id'],axis=1)

In [ ]:
data_model.to_csv("data_for_model.csv",index=False)